## 教師データ収集用スクリプト

衝突回避を学習するためのデータを収集します。
jetbotは走りながらカメラから画像を取り込み、画像をもとに次の行動を決定します。
このタスクでjetbotに必要なことは、障害物がなければまっすぐ進み、障害物があれば左折して回避することの二つです。
よって、「障害物がない写真」、「障害物がある写真」の2パターンを用意する必要があります。
このスクリプトでは、実際にjetbotを遠隔操作で走らせ、直進する(=障害物がない)か左折する(=障害物がある)かを選択します。
直進中に撮られた写真には直進のラベルがつけられ、左折中に撮られた写真には左折のラベルがつけられます。
AIはこのデータを使って、どれが直進していい画像なのかを学習することになります。

AIに使うニューラルネットワークの入力サイズに合わせて、カメラの画像を224px×224pxで保存します。

In [1]:
import traitlets
import ipywidgets.widgets as widgets
from IPython.display import display
from jetbot import Camera, bgr8_to_jpeg

camera = Camera.instance(width=224, height=224)

image = widgets.Image(format='jpeg', width=224, height=224)  # this width and height doesn't necessarily have to match the camera

camera_link = traitlets.dlink((camera, 'value'), (image, 'value'), transform=bgr8_to_jpeg)


画像を保存するフォルダを設定します。
障害物のない画像は`dataset/free`に、障害物のある画像は`dataset/blocked`に保存します。

In [2]:
import os

blocked_dir = 'dataset/blocked'
free_dir = 'dataset/free'

# we have this "try/except" statement because these next functions can throw an error if the directories exist already
try:
    os.makedirs(free_dir)
    os.makedirs(blocked_dir)
except FileExistsError:
    print('Directories not created becasue they already exist')

操作ボタンを定義します。
前進、左折、そして止まるボタンを作っています。

In [3]:
button_layout = widgets.Layout(width='128px', height='48px')
free_button = widgets.Button(description='forward', button_style='success', layout=button_layout)
blocked_button = widgets.Button(description='left', button_style='danger', layout=button_layout)
stop_button = widgets.Button(description='stop', button_style='success', layout=button_layout) 
free_count = widgets.IntText(layout=button_layout, value=len(os.listdir(free_dir)))
blocked_count = widgets.IntText(layout=button_layout, value=len(os.listdir(blocked_dir)))

各フォルダに写真を保存するメソッドを定義しています。

In [4]:
from uuid import uuid1

def save_snapshot(directory):
    image_path = os.path.join(directory, str(uuid1()) + '.jpg')
    with open(image_path, 'wb') as f:
        f.write(image.value)

def save_free():
    global free_dir, free_count
    save_snapshot(free_dir)
    free_count.value = len(os.listdir(free_dir))
    
def save_blocked():
    global blocked_dir, blocked_count
    save_snapshot(blocked_dir)
    blocked_count.value = len(os.listdir(blocked_dir))

モータを制御するため、Robotインスタンスを生成します。

In [5]:
from jetbot import Robot
robot = Robot()

ボタンを押したときのjetbotの挙動を定義します。
前進の時は前進し続けながら一定間隔で画像を保存します。
左折の時は、写真を撮り、少し左へ方向転換して止まります。
これは、直進の時はあまり操作が必要ないので効率性を重視し、障害物を回避するときには慎重に操作する必要があるので確実性を重視したためです。

In [6]:
import time

isFree = False
isRunning = False
stepcount = 0

def set_forward(change):
    global stepcount, isFree, isRunning, robot
    stepcount = 0
    isRunning = True
    isFree = True

def set_left(change):
    global stepcount, isFree, isRunning, robot
    robot.stop()
    save_blocked()
    robot.left(0.4)
    stepcount = 0
    isRunning = True
    isFree = False
            

def update(change):
    global stepcount, isFree, isRunning, robot
    time.sleep(0.001)
    #blocked_count.value = stepcount
    if(isRunning):
        if(isFree):
            if(stepcount % 25 == 0):
                robot.stop()
                save_free()
                robot.forward(0.4)
        else:
            if(stepcount % 6 == 5):
                isRunning = False
                robot.stop()
        stepcount += 1
        
    
def stop(change):
    global isRunning, robot
    isRunning = False
    robot.stop()
    
free_button.on_click(set_forward)
blocked_button.on_click(set_left)
stop_button.on_click(stop)

UIを表示させます。UIを構成するのはカメラからのイメージと操作パネルです。
操作パネル上段には、直進ボタンと直進フォルダに保存された画像の数が表示されます。
また、中段には左折ボタンと左折フォルダに保存された画像の数を表示されます。
最後に止まるボタンを表示します。

In [7]:
display(image)
display(widgets.HBox([free_button, free_count]))
display(widgets.HBox([blocked_button, blocked_count]))
display(stop_button)

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x02\x01\x0…

Button(button_style='success', description='stop', layout=Layout(height='48px', width='128px'), style=ButtonSt…

カメラに先ほど定義したメソッドを紐づけます。
カメラの画像が更新されるたびにupdateメソッドが呼ばれることになります。
これを実行することで、ボタンを押すとjetbotが走り出すようになります。

In [10]:
camera.observe(update, names='value')

以下はデータ収集の終了後に実行する処理です。
updateメソッドとカメラとの関連付けを外します。
また、jetbotを停止させます。

In [11]:
camera.unobserve(update, names='value')
robot.stop()

datasetフォルダの中身をzipファイルにして保存し、ダウンロード可能にします。
jetbot上で学習も行う場合は不要です。

In [12]:
!zip -r -q dataset.zip dataset